# Coursework 1: Finite Fields

[v] By tick the checkbox, we hereby declare that this coursework report is our own and autonomous work. We have acknowledged all material and sources used in its preparation, including books, articles, reports, lecture notes, internet software packages, and any other kind of document, electronic or personal communication. This work has not been submitted for any other assessment.

## 1.1 Integers (30%)

### 1.1.1 Check a prime number

Write a function `isprime` which checks whether a positive integer is a prime or not. Test your function using at least five different numbers (some primes some not) less than 100. 

In [99]:
function isprime(num)
    if num == 2 || num == 3
        return true
    end
    if num < 2 || num % 2 == 0
        return false
    end
    if num < 9
        return true
    end
    if num % 3 == 0
        return false
    end
    r = trunc(Int, num ^ 0.5)
    f = 5
    while f <= r
        if num % f == 0
            return false
        end
        if num % (f + 2) == 0
            return false
        end
        f+= 6
    end
    return true
end
println("13: ", isprime(13)) #true
println("48: ", isprime(48)) #false
println("53: ", isprime(53)) #true
println("97: ", isprime(97)) #true
println("73: ", isprime(73)) #true
println("63: ", isprime(63)) #false
println("3: ", isprime(3)) #true

13: true
48: false
53: true
97: true
73: true
63: false
3: true


### 1.1.2 Extended Euclidean algorithm for integers

1. Get familiar with the function `gcdx` that comes with Julia. 
2. Mimic the input and output style of `gcdx` and write your own function `my_gcdx` for the same functionality.
3. Test your function using at least 5 different instances. 

In [100]:
function my_gcdx(a, b)
    x1, y1, z1 = a, 1, 0
    x2, y2, z2 = b, 0, 1
    while x2 != 0
      q = x1 ÷ x2
      x1, x2 = x2, x1 % x2
      y1, y2 = y2, y1 - q * y2
      z1, z2 = z2, z1 - q * z2
    end
    return x1, y1, z1
  end
println(my_gcdx(42, 5) == gcdx(42, 5))  
println(my_gcdx(9, 99) == gcdx(9, 99))
println(my_gcdx(8, 84) == gcdx(8, 84))
println(my_gcdx(2406, 654) == gcdx(2406, 654))
println(my_gcdx(508, 8) == gcdx(508, 8))

true
true
true
true
true


### 1.1.3 Modular arithmetic 

Let $p=811$, $x=3$, and $n=789$. Compute $x^n \equiv a ~\text{mod}~p$. Show the necessary steps for the computations. 

In [101]:
function mod_computation(p, x, n)
    a = x
    for _ in 2:n # (x ^ n) % p == (x % p * x % p * ...) % p
        a = mod(a * x, p)
    end
    return a
end
println("a = ", mod_computation(811, 3, 789)) # https://www.wolframalpha.com/input/?i=3+%5E+789+mod+811

a = 188


## 1.2 Polynomials (50%)

Let $p$ be a prime number. Consider the polynomial ring $\mathbb{F}_p[x]$. 

### 1.2.1 Polynomial division

Design and write a function `polydiv' which calculates the quotient $q(x)$ and the remainder $r(x)$ from $f(x) / g(x)$ so that 
1. $f(x) = q(x) g(x) + r(x)$ where $\text{deg}(r(x)) < \text{deg}(g(x))$; 
2. When $r(x) \ne 0$, $r(x)$ is a monic polynomial; 
3. Clear documentation to allow other people to know how to input $f(x)$ and $g(x)$. 
4. Run 5 different tests with simple cases. For example, $p=2$ and $p=3$, $\text{deg}(f) = 3$ and $\text{deg}(f) = 4$. 

In [9]:
function getDegree(p)
    degreeCandidate = -1
    for i in 1:(size(p)[2])
       if (p[i] != 0)
        degreeCandidate = i - 1
       end 
    end
    degree = degreeCandidate
    return degree
end

function rightShift(p, shiftAmount)
    pInner = copy(p)
    for q in 1:shiftAmount
        for i in (size(p)[2]):-1:2
            pInner[i] = pInner[i-1]
        end
        pInner[1] = 0
    end
    return pInner
end
function getMultInverse(num, p)
    numR = convert(Int, num)
    pR = convert(Int, p)
    return mod(gcdx(numR, pR)[2], p)
end
function polydiv(f,g,p)
    q = Matrix{Int}(undef, 1, size(f)[2])
    fill!(q, 0.)
    gTemp = rightShift(g, getDegree(f)-getDegree(g))
    while(getDegree(f) >= getDegree(g))
        gTemp = rightShift(g, getDegree(f)-getDegree(g))
        q[(getDegree(f)-getDegree(g))+1] = mod((getMultInverse(gTemp[getDegree(gTemp) + 1], p) * f[(getDegree(f))+1]), p)
        gTemp *= q[(getDegree(f)-getDegree(g)) + 1]
        f -= gTemp
        i = 1
        for coeff in f
            f[i] = mod(coeff, p)
            i = i + 1
        end
    end
    r = f
    return (q,r)
end

#Input polynomial as array, with lowest index representing the coeffecient of lowest degree. [1 2 0 4] = 1 + 2x + 4x^3
#Both polynomials must be inputted as arrays of equal length, using coeffecients of zero when needed
println(polydiv([3 3], [2 0], 5), "    Answer = 4x+4 remainder 0") #3x+3 / 2 in F5
println(polydiv([1 0 1 1],[0 1 1 0], 2), "    Answer = x remainder 1") #x^3+x^2+1 / x^2+x in F2
println(polydiv([1 1 0 1 1],[1 0 1 0 0], 2), "    Answer = x^2+x+1 remainder 0") #x^4+x^3+x+1 / X^2+1 in F2
println(polydiv([1 2 1 2],[2 0 1 0], 3), "    Answer = 2x+1 remainder x+2") #2x^3+x^2+2x+1 / x^2+2 in F3
println(polydiv([0 2 2 0 1],[0 1 2 0 0],3), "    Answer = 2x^2+2x remainder 2x") #x^4+2x^2+2x / 2x^2+x in F3


([4 4], [0 0])    Answer = 4x+4 remainder 0
([0 1 0 0], [1 0 0 0])    Answer = x remainder 1
([1 1 1 0 0], [0 0 0 0 0])    Answer = x^2+x+1 remainder 0
([1 2 0 0], [2 1 0 0])    Answer = 2x+1 remainder x+2
([0 2 2 0 0], [0 2 0 0 0])    Answer = 2x^2+2x remainder 2x


### 1.2.2 Extended Euclidean algorithm for polynomials

Design and write a function `gcdx_poly` to compute the GCD of $f(x)$ and $g(x)$, and their Bezout coefficients. Run 5 different tests to demonstrate the correctness of your function.

In [26]:
function poly_multiply(f, g, p)
    m = zeros(1, size(f)[2])
    for i in 1:getDegree(f)+1
        for j in 1:getDegree(g)+1
            m[(i+j)-1] = mod(m[(i+j)-1] + f[i]*g[j], p)
        end
    end
    return m    
end

function gcdx_poly(f, g, p)
    x1, y1, z1 = f, zeros(1, size(f)[2]), zeros(1, size(f)[2])
    x2, y2, z2 = g, zeros(1, size(f)[2]) , zeros(1, size(f)[2])
    y1[1], z2[1] = 1, 1
    while x2 != zeros(1, size(x2)[2])
        temp = x2  
        q, x2 = polydiv(x1, x2, p)
        x1 = temp 
        y1, y2 = y2, y1-poly_multiply(q,y2, p)
        z1, z2 = z2, z1-poly_multiply(q,z2, p)
        for i in 1:size(y2)[2]
            y2[i] = mod(y2[i], p)
            z2[i] = mod(z2[i], p)
        end
    end
    divider = getMultInverse(x1[getDegree(x1)+1],p)
    for i in 1:length(x1)
        x1[i] = mod(divider * x1[i], p)
        y1[i] = mod(divider * y1[i], p)
        z1[i] = mod(divider * z1[i], p)
    end
    return x1, y1, z1
end

#Answers for the gcd is given as a monic polynomial
println(gcdx_poly([1 1 1 0 1], [0 1 1 0 0], 5)) #gcdx of x^4+x^2+x+1 and x^2+x in F5. Answer = 1 with Bezout's coeffecients 3x+1 and 2x^3+2x^2+1
println(gcdx_poly([1 0 2 1], [2 1 0 0], 3)) #gcdx of x^2+2x^2+1 and x+2 in F3. Answer = 1 with Bezout's coeffecients 1 and 2x^2
println(gcdx_poly([0 1 0 2 1],[2 0 1 0 0], 3)) #gcdx of x^4+2x^3+x and x^2+2 in F4. Answer = 1 with Bezout's coeffecients 1 and 2x^2+x+2
println(gcdx_poly([0 0 1 1],[1 1 0 0], 2)) #gcdx of x^3+x^2 and x+1 in F2. Answer = x+1 with Bezout's coeffecients 0 and 1
println(gcdx_poly([1 1 2 3],[0 2 4 0], 5)) #gcdx of 3x^3+2x^2+x+1 and 4x^2+2x in F5. Answer = x+3 with Bezout's coeffecients 3 and 4x+4


([1 0 0 0 0], [1.0 3.0 0.0 0.0 0.0], [1.0 0.0 2.0 2.0 0.0])
([1 0 0 0], [1.0 0.0 0.0 0.0], [0.0 0.0 2.0 0.0])
([1 0 0 0 0], [1.0 0.0 0.0 0.0 0.0], [2.0 1.0 2.0 0.0 0.0])
([1 1 0 0], [0.0 0.0 0.0 0.0], [1.0 0.0 0.0 0.0])
([3 1 0 0], [3.0 0.0 0.0 0.0], [4.0 4.0 0.0 0.0])


### 1.2.3 Irreducible polynomial

Design and write a function `is_irreducible` to check whether $f(x) \in \mathbb{F}_p[x]$ is irreducible or not. Run 5 tests to demonstrate the correctness of your function. 

In [34]:
function generateAllCombs(coeffRing, data, ringSize, index, numCoeffs, f)
    if (index == numCoeffs + 1)
        dataIsConstant = true
        dataIsSameOrderAsF = true
        
        for i in 2:numCoeffs
           if (data[i] != 0)
               dataIsConstant = false  
            end
        end
        if(data[numCoeffs] == 0)
            dataIsSameOrderAsF = false
        end
        if(!dataIsConstant && !dataIsSameOrderAsF)
            q, r = polydiv(f, data, ringSize)
            zeroMatrix = Matrix{Float64}(undef, 1, numCoeffs)
            fill!(zeroMatrix, 0.)
            if(r == zeroMatrix)
                return false
            else
               return true 
            end
        else
            return true
        end
    end
    
    i = 1
    irreducible = true
    while(i <= ringSize)
        data[index] = coeffRing[i]
        irreducibleCurrent = generateAllCombs(coeffRing, data, ringSize, index + 1, numCoeffs, f)
        irreducible = irreducible && irreducibleCurrent
        i = i + 1
    end
    return irreducible
end
function is_irreducible(f, p)
    degree = getDegree(f)
    intRing = Matrix{Int}(undef, 1, p)
    for i in 1:p
       intRing[i] = i-1 
    end
    candidateDivisor = Matrix{Int}(undef, 1, degree+1)
    return generateAllCombs(intRing, candidateDivisor, p, 1, degree+1, f)
end

println(is_irreducible([1 1 0 0 1], 2)) #x^4+x+1 in F2
println(is_irreducible([1 0 1], 5)) #x^2+1 in F5
println(is_irreducible([1 0 1], 3)) #x^2+1 in F3
println(is_irreducible([1 2 0 5], 7)) #5x^3+2x+1 in F7
println(is_irreducible([1 2 0 3], 7)) #3x^3+2x+1 in F7    

true
false
true
false
true


## 1.3 Primitive element (20%)

### 1.3.1 Find primitive elements 

Consider the finite field $\mathbb{F}_p$. Design and write a function `find_primitive` to find all primitive elements in $\mathbb{F}_p$. Test your function with a prime number $20 < p < 50$. 

In [9]:
function find_primitive(p)
    primeEls = Int[]
    for a in 2:p-1
        test = a
        for t in 2:p-1
            test = mod(test * a, p)
            if test == 1
                if t == p-1
                    append!(primeEls, a)
                end
                break
            end
        end
    end
    return primeEls
end

find_primitive(47)


22-element Vector{Int64}:
  5
 10
 11
 13
 15
 19
 20
 22
 23
 26
 29
 30
 31
 33
 35
 38
 39
 40
 41
 43
 44
 45

### 1.3.2 Calculate primitive elements

Now consider the finite field $\mathbb{F}_{32}$. Without programming, can you find all the primitive elements in $\mathbb{F}_{32}$? If so, explain your approach and its rationality. 

$\mathbb{F}_{32}$ is a well defined field. As 32 = 2^5, this is a field of 31 non-zero polynomials. There are 30 primitive elements in this finite field. q = 32 and so q-1 = 31. 31 is a prime number which means that for any non-zero polynomial, its order must divide 31. As the only numbers that divide 31 are 1 and 31, all non-zero elements that are not 1 (of which there are 30) must have an order of 31, which makes them a primitive element. So all non-zero polynomials that are not 1 are primitive elements of $\mathbb{F}_{32}$.

## Highlight

Please list a couple of highlights of your coursework that may impress your markers.

-Our implementation of inputting polynomials allows for polynomials of any size to be easily entered. 

-Both implementations of the Extended Euclidean algorithm are memory efficient, without needed to store large vectors of all values used to calculate the Bezout's coefficients. The coefficients are calculated continuously as the algorithm progresses.

